In [1]:
import pandas as pd
df1 = pd.read_csv('./4차년도.csv', encoding='CP949')
df2 = pd.read_csv('./5차년도.csv', encoding='CP949')
df3 = pd.read_csv('./5차년도_2차.csv', encoding='CP949') 
data = pd.concat([df1, df2, df3], ignore_index=True)

# 데이터 shape
print(data.shape)
# 전체 감정의 수
emotion_counts = data['상황'].value_counts()
emotion_counts

(43991, 15)


상황
sad          11152
anger         8372
disgust       4660
happiness     4548
fear          4131
angry         3263
neutral       3262
sadness       2848
surprise      1755
Name: count, dtype: int64

##### 아마 5명이 해당 음성에 대해 주관적으로 평가한 것 같음 -> 구체적으로 진행하면 복잡해지기에 우선 상황을 기준을 해당 감정이라고 생각하고 진행 

In [2]:
# 두려움은 약 7%
fear_data = data[data['상황']=='fear']
print(len(fear_data)/len(data))

0.09390557159418972


In [3]:
data['1번 감정'] = data['1번 감정'].apply(str.lower)
data['2번 감정'] = data['2번 감정'].apply(str.lower)
data['3번 감정'] = data['3번 감정'].apply(str.lower)
data['4번 감정'] = data['4번 감정'].apply(str.lower)
data['5번 감정'] = data['5번 감정'].apply(str.lower)

In [4]:
def get_keys(dic): #returns a key for max values in dic
  key_list = list(dic.keys())
  val_list = list(dic.values())
  pos = val_list.index(max(val_list))
  return key_list[pos]
  
final_label = []
for i in range(len(data)):
  sentiments = {'angry':0, 'sadness':0, 'happiness':0, 'fear': 0, 'disgust':0, 'surprise':0, 'neutral':0}
  sentiments[data.iloc[i]['1번 감정']] += data.iloc[i]['1번 감정세기']
  sentiments[data.iloc[i]['2번 감정']] += data.iloc[i]['2번 감정세기']
  sentiments[data.iloc[i]['3번 감정']] += data.iloc[i]['3번 감정세기']
  sentiments[data.iloc[i]['4번 감정']] += data.iloc[i]['4번감정세기']
  sentiments[data.iloc[i]['5번 감정']] += data.iloc[i]['5번 감정세기']

  final_label.append(get_keys(sentiments))
  
final_label_df = pd.DataFrame(final_label, columns=['상황'])
new_data = pd.concat([data[['wav_id', '발화문']], final_label_df], axis = 1)
new_data

,wav_id,발화문,상황
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",angry
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,angry
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,angry
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,sadness
4,5e27f90b5807b852d9e0157b,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sadness
...,...,...,...
43986,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,sadness
43987,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,sadness
43988,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,sadness
43989,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,sadness


### 감정 label마다 감정 세기를 더하여 감정 세기가 제일 큰 감정을 해당 발화문의 감정 label로 설정

In [5]:
emotion_counts = new_data['상황'].value_counts()
emotion_counts

상황
sadness      21264
angry        10425
happiness     5103
fear          3071
disgust       2692
surprise      1436
Name: count, dtype: int64

### 기존 1386에서 1152로 fear 감소한 것을 알 수 있음

In [ ]:
# Separate fear and non-fear samples
fear_samples = new_data[new_data['상황'] == 'fear']
non_fear_samples = new_data[new_data['상황'] != 'fear']

# Randomly sample 1152 non-fear instances
non_fear_sampled = non_fear_samples.sample(n=3071, random_state=42)

# Combine fear and sampled non-fear instances into a new balanced dataset
balanced_data = pd.concat([fear_samples, non_fear_sampled], ignore_index=True)
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the balanced dataset's distribution and size
balanced_data_info = balanced_data['상황'].value_counts(), balanced_data.shape
balanced_data_info



(상황
 fear         3071
 sadness      1632
 angry         757
 happiness     364
 disgust       202
 surprise      116
 Name: count, dtype: int64,
 (6142, 3))

In [18]:
balanced_data

,wav_id,발화문,상황
0,5f653d94111dfd48d40fc9d4,창문으로 집 밖을 봤는데 집 주변까지 물이 차올라서 밖으로 나갈 수가 없어.,fear
1,5e33cf195807b852d9e0585f,이주일전쯤에 한번 얘기해봤어.,angry
2,5e37cf0b7995ef170fc0f373,"나 내일 약속 있는데, 그대로 가도 될까?",fear
3,5f6f5baf9e04b149046cbe20,태풍에 장마가 겹쳐서 엉망이 돼버렸어. 나가지도 못하고.,sadness
4,5e33dc695807b852d9e05c9a,갑자기 그렇게 됐네.,sadness
...,...,...,...
6137,5e3647b3c8c25f16cd144b34,"우리 아빠, 어제 또 다쳤어!",angry
6138,5e40ef6448600202f726fa82,그냥 길을 걷고 있는데 집에도 못 가겠어. 부모님이 뭐라고 하실지 몰라서.,sadness
6139,5f6d5b07f8fac448cc0a5fc9,아니 아까 해피를 산책시키다가 깜짝 놀랐어.,surprise
6140,5f98a721111dfd48d40ff6e6,아까는 해피때문에 엄청 놀랐는데 지금은 진정되었어.,surprise


In [20]:
from sklearn.model_selection import train_test_split

# 불안과 비불안 데이터를 각각 분리
fear_samples = balanced_data[balanced_data['상황'] == 'fear']
non_fear_samples = balanced_data[balanced_data['상황'] != 'fear']

# 각 데이터를 90:10으로 분할
fear_train, fear_test = train_test_split(fear_samples, test_size=0.1, random_state=42)
non_fear_train, non_fear_test = train_test_split(non_fear_samples, test_size=0.1, random_state=42)

# Train과 Test 데이터셋을 결합하여 1:1 비율 유지
train_data = pd.concat([fear_train, non_fear_train], ignore_index=True)
train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)
test_data = pd.concat([fear_test, non_fear_test], ignore_index=True)
test_data = test_data.sample(frac=1, random_state=42).reset_index(drop=True)

# 각각의 데이터셋을 CSV 파일로 저장
train_data.to_csv('Total_balanced_train_data.csv', index=False, encoding='CP949')
test_data.to_csv('Total_balanced_test_data.csv', index=False, encoding='CP949')

# 결과 확인
print("Train 데이터셋 크기:", train_data.shape)
print("Test 데이터셋 크기:", test_data.shape)

Train 데이터셋 크기: (5526, 3)
Test 데이터셋 크기: (616, 3)


In [31]:
import os
import pandas as pd
import shutil

# 경로 설정
test_data_folder = './Total_balanced_train_data.csv'
wav_folder = './4차년도_2차'
test_folder = './Total_train'

# train_data 폴더가 없다면 생성
os.makedirs(test_folder, exist_ok=True)

# CSV 파일에서 wav_id 열을 불러오기
data = pd.read_csv(test_data_folder, encoding='CP949')
wav_ids = data['wav_id'].astype(str).tolist()  # 모든 값을 문자열로 변환

# 파일 옮기기
for wav_id in wav_ids:
    wav_file = f"{wav_id}.wav"
    wav_path = os.path.join(wav_folder, wav_file)
    
    # 파일이 존재하면 train_data 폴더로 이동
    if os.path.exists(wav_path):
        shutil.move(wav_path, test_folder)

print("선택된 파일들이 ./test_data 폴더로 이동되었습니다.")


선택된 파일들이 ./test_data 폴더로 이동되었습니다.
